In [24]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [25]:
import seaborn as sns

In [26]:
df=sns.load_dataset('iris')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [27]:
df.isnull().sum()

sepal_length    0
sepal_width     0
petal_length    0
petal_width     0
species         0
dtype: int64

In [41]:
X=df.drop(labels=['species'],axis=1)
y=df['species']

In [43]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [44]:
numerical_cols = ['sepal_length'	,'sepal_width'	,'petal_length',	'petal_width']

In [45]:
num_pipeline=Pipeline(
    steps=[
        ('imputer',SimpleImputer(strategy='median')), ##Missing Values
        ('scaler',StandardScaler()) ## feature Scaling
    ]

)

In [46]:
preprocessor=ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols)
])

preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['sepal_length', 'sepal_width', 'petal_length',
                                  'petal_width'])])

In [47]:
from sklearn.ensemble import RandomForestClassifier

In [48]:
models={
    'Random Forest':RandomForestClassifier(),
    'Logistic Regression':LogisticRegression()
}

In [49]:
from sklearn.metrics import accuracy_score

In [51]:
def evaluate_model(X_train,y_train,X_test,y_test,models):
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        model.fit(X_train,y_train)

        y_test_pred =model.predict(X_test)
        test_model_score = accuracy_score(y_test,y_test_pred)
        report[list(models.keys())[i]] =  test_model_score
    return report

In [52]:
evaluate_model(X_train,y_train,X_test,y_test,models)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'Random Forest': 1.0, 'Logistic Regression': 1.0}

In [53]:
classfier=RandomForestClassifier()

In [54]:
params={'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['gini','entropy']
              }

In [55]:
from sklearn.model_selection import GridSearchCV

In [57]:
cv=GridSearchCV(classfier,param_grid=params,scoring='accuracy',cv=5,verbose=3)
cv.fit(X_train,y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END criterion=gini, max_depth=3, n_estimators=100;, score=0.958 total time=   0.3s
[CV 2/5] END criterion=gini, max_depth=3, n_estimators=100;, score=1.000 total time=   0.3s
[CV 3/5] END criterion=gini, max_depth=3, n_estimators=100;, score=0.875 total time=   0.3s
[CV 4/5] END criterion=gini, max_depth=3, n_estimators=100;, score=1.000 total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=3, n_estimators=100;, score=0.958 total time=   0.3s
[CV 1/5] END criterion=gini, max_depth=3, n_estimators=200;, score=0.958 total time=   0.6s
[CV 2/5] END criterion=gini, max_depth=3, n_estimators=200;, score=1.000 total time=   0.3s
[CV 3/5] END criterion=gini, max_depth=3, n_estimators=200;, score=0.833 total time=   0.3s
[CV 4/5] END criterion=gini, max_depth=3, n_estimators=200;, score=1.000 total time=   0.3s
[CV 5/5] END criterion=gini, max_depth=3, n_estimators=200;, score=0.958 total time=   0.3s
[CV 1/5] END crite

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [3, 5, 10, None],
                         'n_estimators': [100, 200, 300]},
             scoring='accuracy', verbose=3)

In [58]:
cv.best_params_

{'criterion': 'gini', 'max_depth': 3, 'n_estimators': 100}

In [59]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

rf_classifier = RandomForestClassifier()
lr_classifier = LogisticRegression()
voting_classifier = VotingClassifier(
    estimators=[('rf', rf_classifier), ('lr', lr_classifier)],
    voting='soft'
)

voting_classifier.fit(X_train, y_train)
accuracy = cross_val_score(voting_classifier, X_train, y_train, cv=5).mean()
print("Accuracy:", accuracy)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Accuracy: 0.9583333333333334


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
